In [ ]:
import requests
import chardet

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

counter = 0
c2 = 0

OUTPUT_FILE = "../data/address_ies_with_lat_long.csv"
INPUT_FILE = "../data/address_ies.csv"

with open(INPUT_FILE, 'rb') as f:
    result = chardet.detect(f.read())
encoding = result['encoding']

with open(INPUT_FILE, 'r', encoding=encoding) as fp:  # Abrindo o arquivo com encoding detectado
    with open(OUTPUT_FILE, "w") as outp:
        outp.writelines("CO_IES,LAT,LONG\n")  # Escrevendo o cabeçalho no arquivo de saída
        next(fp) # Ignorando a primeira linha do cabeçalho
        for line in fp:
            # Contador de requests
            if counter < 100000:
                line_list = line.split(';')

                # Validando o número do endereço
                numero = line_list[1] if is_number(line_list[1]) else ""

                # Construindo a string de endereço na ordem correta e formatada
                msg = f"{numero},{line_list[2]},{line_list[3]},{line_list[4]},{line_list[5]},Brasil"
                msg = msg.replace(",", "%2C")  # Substituindo vírgulas por %2C
                msg = msg.replace(" ", "+")  # Substituindo espaços por +

                get_str = ('https://nominatim.openstreetmap.org/search?q=' + msg + '&format=json&addressdetails=1&limit=1')  # Construindo a URL

                print(get_str)
                headers = {
                    'User-Agent': 'Marlos Piccinelli <mpiccinelli@gmail.com>'
                }
                response = requests.get(get_str, headers=headers)

                if response.status_code == 200:
                    resp_json_payload = response.json()
                    
                    # Extraindo as coordenadas diretamente do primeiro elemento da lista
                    lat = '' if not resp_json_payload else resp_json_payload[0]['lat']
                    lng = '' if not resp_json_payload else resp_json_payload[0]['lon']
                    
                    counter = counter + 1
                else:
                    lat = ""
                    lng = ""

                outp.writelines(line_list[0] + "," + lat + "," + lng + "\n")  # Gravando a linha com as coordenadas
                c2 = c2 +1